# 2.2 Pandas, Basic Mapping

This section aims to provide new skills in python to handle structured "table" data. 

Learning outcome:
-   Manipulation of data frames (describing, filtering, ...) 
-   Learn about Lambda functions
-   Intro to datetime objects
-   Plotting data from data frames (histograms and maps)
-   Introduction to Plotly
-   Introduction to CSV & Parquet


We will work on several structured data sets: sensor metadata, seismic data product (earthquake catalog).

First, we import all the modules we need:

In [1]:
import numpy as np
import pandas as pd
import io
import requests
import time
from datetime import datetime, timedelta

import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'vscode' # writes as standalone html, 
# try notebook, jupyterlab, png, vscode, iframe


## Creating a data frame from standard text files

The python package pandas is very useful to read csv files, but also many text files that are more or less formated as one observation per row and one column for each feature.

As an example, we are going to look at the list of seismic stations from the Northern California seismic network, available [here](http://ncedc.org/ftp/pub/doc/NC.info/NC.channel.summary.day):



In [2]:
url = 'http://ncedc.org/ftp/pub/doc/NC.info/NC.channel.summary.day'

The function read_csv is used to open and read your text file. In the case of a well formatted csv file, only the name of the file needs to be entered:

     data = pd.read_csv('my_file.csv')

However, many options are available if the file is not well formatted. See more in this [tutorial](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html):



In [3]:
# this gets the file linked in the URL page and convert it to a string
s = requests.get(url).content 
# this will convert the string, decode it , and make it a table
data = pd.read_csv(io.StringIO(s.decode('utf-8')), header=None, skiprows=2, sep='\s+', usecols=list(range(0, 13)))
# because columns/keys were not assigned, assign them now
data.columns = ['station', 'network', 'channel', 'location', 'rate', 'start_time', 'end_time', 'latitude', 'longitude', 'elevation', 'depth', 'dip', 'azimuth']

Let us look at the data. They are now stored into a pandas dataframe. Read the top of the table:

In [4]:
data.head()

,station,network,channel,location,rate,start_time,end_time,latitude,longitude,elevation,depth,dip,azimuth
0,AAR,NC,EHZ,--,100.0,"1984/01/01,00:00:00","1987/05/01,00:00:00",39.27594,-121.02696,911.0,0.0,-90.0,0.0
1,AAR,NC,EHZ,--,100.0,"1987/05/01,00:00:00","2006/01/04,19:19:00",39.27594,-121.02696,911.0,0.0,-90.0,0.0
2,AAR,NC,SHZ,--,20.0,"1994/11/28,00:00:00","2006/01/04,19:19:00",39.27594,-121.02696,911.0,0.0,-90.0,0.0
3,AAS,NC,EHZ,--,100.0,"1984/11/27,18:45:00","1987/05/01,00:00:00",38.43014,-121.10959,31.0,0.0,-90.0,0.0
4,AAS,NC,EHZ,--,100.0,"1987/05/01,00:00:00","2021/08/13,16:50:00",38.43014,-121.10959,31.0,0.0,-90.0,0.0


There are two aways of looking at a particular column:

In [5]:
data.station

0        AAR
1        AAR
2        AAR
3        AAS
4        AAS
        ... 
7151     WMP
7152     WMP
7153     WMP
7154     WMP
7155    WWVB
Name: station, Length: 7156, dtype: object

In [6]:
# or like this:
data['station']

0        AAR
1        AAR
2        AAR
3        AAS
4        AAS
        ... 
7151     WMP
7152     WMP
7153     WMP
7154     WMP
7155    WWVB
Name: station, Length: 7156, dtype: object

If we want to look at a given row or column, and we know its index, we can do:

In [7]:
data.iloc[0]

station                       AAR
network                        NC
channel                       EHZ
location                       --
rate                        100.0
start_time    1984/01/01,00:00:00
end_time      1987/05/01,00:00:00
latitude                 39.27594
longitude              -121.02696
elevation                   911.0
depth                         0.0
dip                         -90.0
azimuth                       0.0
Name: 0, dtype: object

In [8]:
data.iloc[:, 0]

0        AAR
1        AAR
2        AAR
3        AAS
4        AAS
        ... 
7151     WMP
7152     WMP
7153     WMP
7154     WMP
7155    WWVB
Name: station, Length: 7156, dtype: object

If we know the name of the column, we can do:

In [9]:
data.loc[:, 'station']

0        AAR
1        AAR
2        AAR
3        AAS
4        AAS
        ... 
7151     WMP
7152     WMP
7153     WMP
7154     WMP
7155    WWVB
Name: station, Length: 7156, dtype: object

We can also access a single value within a column:

In [10]:
data.loc[0, 'station']

'AAR'

## Mapping usiny Plotly

Now we will plot the station locations on a map using the Plotly package. More tutorials on [Plotly](https://plotly.com/). Input of the function in the function is self-explanatory and typical of Python's function. The code [documentation](https://plotly.com/python/scatter-plots-on-maps/) of Plotly scatter_geo lists the variables.

In [11]:
fig = px.scatter_geo(data,
                     lat='latitude',lon='longitude', 
                     range_color=(6,9),
                     height=800, width=800,
                     hover_name="station",
                     hover_data=['network','station','channel','rate']);
fig.update_geos(resolution=110, showcountries=True,projection_type="orthographic")

In [12]:
fig = px.scatter_mapbox(data,
                     lat='latitude',lon='longitude', 
                     range_color=(6,9),mapbox_style="carto-positron",
                     height=800, width=800,
                     hover_name="station",
                     hover_data=['network','station','channel','rate']);
fig.update_layout(title="Northern California Seismic Network")
fig.show()

## Pandas: data selection
We can filter the data with the value taken by a given column:

In [13]:
data.loc[data.station=='KCPB']

,station,network,channel,location,rate,start_time,end_time,latitude,longitude,elevation,depth,dip,azimuth
3407,KCPB,NC,BHE,--,50.0,"1999/08/03,00:00:00","2000/06/06,16:00:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
3408,KCPB,NC,BHE,--,50.0,"2000/06/06,16:00:00","2002/01/24,23:50:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
3409,KCPB,NC,BHE,--,50.0,"2002/01/24,23:50:00","2002/10/16,23:59:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
3410,KCPB,NC,BHE,--,20.0,"2002/10/17,00:00:00","2006/01/24,18:00:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
3411,KCPB,NC,BHN,--,50.0,"1999/08/03,00:00:00","2000/06/06,16:00:00",39.68631,-123.58242,1261.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3480,KCPB,NC,MNE,--,10.0,"2000/06/06,16:00:00","2000/07/12,00:00:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
3481,KCPB,NC,MNN,--,10.0,"1999/08/03,00:00:00","2000/06/06,16:00:00",39.68631,-123.58242,1261.0,0.0,0.0,0.0
3482,KCPB,NC,MNN,--,10.0,"2000/06/06,16:00:00","2000/07/12,00:00:00",39.68631,-123.58242,1261.0,0.0,0.0,0.0
3483,KCPB,NC,MNZ,--,10.0,"1999/08/03,00:00:00","2000/06/06,16:00:00",39.68631,-123.58242,1261.0,0.0,-90.0,0.0


In [14]:
# Select two stations
data.loc[(data.station=='KCPB') | (data.station=='KHBB')]

,station,network,channel,location,rate,start_time,end_time,latitude,longitude,elevation,depth,dip,azimuth
3407,KCPB,NC,BHE,--,50.0,"1999/08/03,00:00:00","2000/06/06,16:00:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
3408,KCPB,NC,BHE,--,50.0,"2000/06/06,16:00:00","2002/01/24,23:50:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
3409,KCPB,NC,BHE,--,50.0,"2002/01/24,23:50:00","2002/10/16,23:59:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
3410,KCPB,NC,BHE,--,20.0,"2002/10/17,00:00:00","2006/01/24,18:00:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
3411,KCPB,NC,BHN,--,50.0,"1999/08/03,00:00:00","2000/06/06,16:00:00",39.68631,-123.58242,1261.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3666,KHBB,NC,LHZ,--,1.0,"2016/04/28,16:56:00","2022/08/09,18:00:00",40.65990,-123.21966,1864.0,0.0,-90.0,0.0
3667,KHBB,NC,LHZ,--,1.0,"2022/08/09,18:00:00","3000/01/01,00:00:00",40.65990,-123.21966,1864.0,0.0,-90.0,0.0
3668,KHBB,NC,LNE,--,1.0,"2015/10/29,21:18:00","2016/04/28,16:56:00",40.65990,-123.21966,1864.0,0.0,0.0,90.0
3669,KHBB,NC,LNN,--,1.0,"2015/10/29,21:18:00","2016/04/28,16:56:00",40.65990,-123.21966,1864.0,0.0,0.0,0.0


In [15]:
# or like this
data.loc[data.station.isin(['KCPB', 'KHBB'])]

,station,network,channel,location,rate,start_time,end_time,latitude,longitude,elevation,depth,dip,azimuth
3407,KCPB,NC,BHE,--,50.0,"1999/08/03,00:00:00","2000/06/06,16:00:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
3408,KCPB,NC,BHE,--,50.0,"2000/06/06,16:00:00","2002/01/24,23:50:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
3409,KCPB,NC,BHE,--,50.0,"2002/01/24,23:50:00","2002/10/16,23:59:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
3410,KCPB,NC,BHE,--,20.0,"2002/10/17,00:00:00","2006/01/24,18:00:00",39.68631,-123.58242,1261.0,0.0,0.0,90.0
3411,KCPB,NC,BHN,--,50.0,"1999/08/03,00:00:00","2000/06/06,16:00:00",39.68631,-123.58242,1261.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3666,KHBB,NC,LHZ,--,1.0,"2016/04/28,16:56:00","2022/08/09,18:00:00",40.65990,-123.21966,1864.0,0.0,-90.0,0.0
3667,KHBB,NC,LHZ,--,1.0,"2022/08/09,18:00:00","3000/01/01,00:00:00",40.65990,-123.21966,1864.0,0.0,-90.0,0.0
3668,KHBB,NC,LNE,--,1.0,"2015/10/29,21:18:00","2016/04/28,16:56:00",40.65990,-123.21966,1864.0,0.0,0.0,90.0
3669,KHBB,NC,LNN,--,1.0,"2015/10/29,21:18:00","2016/04/28,16:56:00",40.65990,-123.21966,1864.0,0.0,0.0,0.0


We can access to a brief summary of the data:

In [16]:
data.station.describe()

count     7156
unique     925
top       KCPB
freq        78
Name: station, dtype: object

In [17]:
data.elevation.describe()

count    7156.000000
mean      663.180729
std       670.157818
min     -1388.000000
25%       161.000000
50%       442.000000
75%       925.000000
max      3680.000000
Name: elevation, dtype: float64

We can perform standard operations on the whole data set:

In [18]:
data.mean()

/var/folders/js/lzmy975n0l5bjbmr9db291m00000gn/T/ipykernel_72479/531903386.py:1: FutureWarning:

The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



rate          87.726781
latitude      37.947330
longitude   -121.488346
elevation    663.180729
depth         15.022834
dip          -41.641979
azimuth       17.686417
dtype: float64

In the case of a categorical variable, we can get the list of possile values that this variable can take:

In [19]:
data.channel.unique()

array(['EHZ', 'SHZ', 'HHE', 'HHN', 'HHZ', 'HNE', 'HNN', 'HNZ', 'LHE',
       'LHN', 'LHZ', 'ACE', 'GAN', 'GEL', 'GLA', 'GLO', 'GNS', 'GPL',
       'LCE', 'LCQ', 'VCO', 'VDT', 'VEC', 'VEI', 'VPB', 'ELE', 'ELN',
       'ELZ', 'SLE', 'SLN', 'SLZ', 'LCL', 'LOG', 'OCF', 'VEA', 'VEP',
       'VFP', 'VKI', 'GST', 'EHE', 'EHN', 'BNE', 'BNN', 'BN1', 'BN2',
       'BN3', 'BV1', 'EP1', 'EP2', 'EP3', 'HDO', 'HN1', 'HN2', 'HN3',
       'HV1', 'SP2', 'SP3', 'BNZ', 'LDO', 'HJ2', 'HJ3', 'HJZ', 'ATT',
       'SHE', 'SHN', 'BHE', 'BHN', 'BHZ', 'LNE', 'LNN', 'LNZ', 'MHE',
       'MHN', 'MHZ', 'MNE', 'MNN', 'MNZ', 'HH2', 'HH3', 'LH2', 'LH3',
       'SP1', 'DP1', 'DP2', 'DP3', 'HLE', 'HLN', 'HLZ', 'XNE', 'XNN',
       'XNZ', 'EH1', 'VM1', 'VM2', 'VM3'], dtype=object)

and get the number of times that each value is taken:

In [20]:
data.station.value_counts()

KCPB     78
JSGB     73
KMPB     72
KHMB     63
CCH1     60
         ..
IRG2E     1
IRG3      1
IRGCH     1
IRGE      1
WWVB      1
Name: station, Length: 925, dtype: int64

There are several ways of doing an operation on all rows of a column. The first option is to use the map function.

If you are not familiar with lambda function in Python, look at:

https://realpython.com/python-lambda/

We will practice a bit lambda functions

In [21]:
# Let's create a function to return the difference between the input and a value

b = 1 # b is declared outside of a function and is a GLOBAL variable

def remove_elevation(x):
    return x-b
print(remove_elevation(3))

2


In [22]:
# now we are creating a function with a local variable
def remove_elevation2(x):
    b2=2 # b is now declared inside the function, is LOCAL, and cannot be called outside.
    return x-b2
print(remove_elevation2(3))
# print(b2)

1


In [23]:
# Now the equivalent in lambda is:
remove_b = lambda x:x-b
 # b was defined above so it is a parameter. x is the variable

In [24]:
remove_b(3)

2

In [25]:
# you can add several variables into lambda functions
remove_anything = lambda x,y:x-y
remove_anything(3,b)

2

In [26]:
data_elevation_mean = data.elevation.mean()
data.elevation.map(lambda p: p - data_elevation_mean)

0       247.819271
1       247.819271
2       247.819271
3      -632.180729
4      -632.180729
           ...    
7151    414.819271
7152    414.819271
7153    414.819271
7154    414.819271
7155   -662.180729
Name: elevation, Length: 7156, dtype: float64

The second option is to use the apply function:

In [27]:
def remean_elevation(row):
    row.elevation = row.elevation - data_elevation_mean
    return row
data.apply(remean_elevation, axis='columns')

,station,network,channel,location,rate,start_time,end_time,latitude,longitude,elevation,depth,dip,azimuth
0,AAR,NC,EHZ,--,100.0,"1984/01/01,00:00:00","1987/05/01,00:00:00",39.27594,-121.02696,247.819271,0.0,-90.0,0.0
1,AAR,NC,EHZ,--,100.0,"1987/05/01,00:00:00","2006/01/04,19:19:00",39.27594,-121.02696,247.819271,0.0,-90.0,0.0
2,AAR,NC,SHZ,--,20.0,"1994/11/28,00:00:00","2006/01/04,19:19:00",39.27594,-121.02696,247.819271,0.0,-90.0,0.0
3,AAS,NC,EHZ,--,100.0,"1984/11/27,18:45:00","1987/05/01,00:00:00",38.43014,-121.10959,-632.180729,0.0,-90.0,0.0
4,AAS,NC,EHZ,--,100.0,"1987/05/01,00:00:00","2021/08/13,16:50:00",38.43014,-121.10959,-632.180729,0.0,-90.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7151,WMP,NC,SHN,--,20.0,"1995/07/02,12:00:00","2002/05/08,22:30:00",35.64059,-118.78570,414.819271,0.0,0.0,0.0
7152,WMP,NC,SHZ,--,20.0,"1995/03/02,19:00:00","1995/07/02,12:00:00",35.64059,-118.78570,414.819271,0.0,-90.0,0.0
7153,WMP,NC,SHZ,--,20.0,"1995/07/02,12:00:00","2002/05/08,22:30:00",35.64059,-118.78570,414.819271,0.0,-90.0,0.0
7154,WMP,NC,SHZ,10,20.0,"1995/07/02,12:00:00","1999/05/11,23:59:00",35.64059,-118.78570,414.819271,0.0,-90.0,0.0


We can also carry out simple operations on columns, provided they make sense.

In [28]:
netsta=(data.network + '.' + data.station)
print(netsta)
print(type(netsta))

0        NC.AAR
1        NC.AAR
2        NC.AAR
3        NC.AAS
4        NC.AAS
         ...   
7151     NC.WMP
7152     NC.WMP
7153     NC.WMP
7154     NC.WMP
7155    NC.WWVB
Length: 7156, dtype: object
<class 'pandas.core.series.Series'>


In [29]:
netsta=(data.network + '.' + data.station).values
print(netsta)
print(type(netsta))

['NC.AAR' 'NC.AAR' 'NC.AAR' ... 'NC.WMP' 'NC.WMP' 'NC.WWVB']
<class 'numpy.ndarray'>


A useful feature is to group the rows depending on the value of a categorical variable, and then apply the same operation to all the groups. For instance, we want to know how many times each station appears in the file:

In [30]:
data.groupby('station').station.count()

station
AAR      3
AAS      3
ABJ      4
ABR      3
ADW      3
        ..
VRC      4
VSP      2
VWB      1
WMP     10
WWVB     1
Name: station, Length: 925, dtype: int64

Or we want to know what is the lowest and the highest elevation for each station:

In [31]:
data.groupby('station').elevation.min()

station
AAR      911.0
AAS       31.0
ABJ      434.0
ABR       -1.0
ADW      228.0
         ...  
VRC     1666.0
VSP     1545.0
VWB     1736.0
WMP     1078.0
WWVB       1.0
Name: elevation, Length: 925, dtype: float64

We can have access to the data type of each column:

In [33]:
data.dtypes

station        object
network        object
channel        object
location       object
rate          float64
start_time     object
end_time       object
latitude      float64
longitude     float64
elevation     float64
depth         float64
dip           float64
azimuth       float64
dtype: object

Here, pandas does not recognize the start_time and end_time columns as a datetime format, so we cannot use datetime operations on them. We first need to convert these columns into a datetime format:

In [34]:
data.start_time.values()

TypeError: 'numpy.ndarray' object is not callable

In [ ]:
type(data['start_time'][0])

str

In [35]:
# Transform column from string into datetime format
startdate = pd.to_datetime(data['start_time'], format='%Y/%m/%d,%H:%M:%S')
data['start_time'] = startdate
print(data['start_time'] )
type(data['start_time'][0])

0      1984-01-01 00:00:00
1      1987-05-01 00:00:00
2      1994-11-28 00:00:00
3      1984-11-27 18:45:00
4      1987-05-01 00:00:00
               ...        
7151   1995-07-02 12:00:00
7152   1995-03-02 19:00:00
7153   1995-07-02 12:00:00
7154   1995-07-02 12:00:00
7155   1984-01-01 00:00:00
Name: start_time, Length: 7156, dtype: datetime64[ns]


pandas._libs.tslibs.timestamps.Timestamp

In [36]:
print(data['end_time'])

0       1987/05/01,00:00:00
1       2006/01/04,19:19:00
2       2006/01/04,19:19:00
3       1987/05/01,00:00:00
4       2021/08/13,16:50:00
               ...         
7151    2002/05/08,22:30:00
7152    1995/07/02,12:00:00
7153    2002/05/08,22:30:00
7154    1999/05/11,23:59:00
7155    1999/03/20,23:59:00
Name: end_time, Length: 7156, dtype: object


In [37]:
# do the same for end times
# Avoid 'OutOfBoundsDatetime' error with year 3000
enddate = data['end_time'].str.replace('3000', '2025')
enddate = pd.to_datetime(enddate, format='%Y/%m/%d,%H:%M:%S')
data['end_time'] = enddate

We can now look when each seismic station was installed:

In [38]:
data.groupby('station').apply(lambda df: df.start_time.min())

station
AAR    1984-01-01 00:00:00
AAS    1984-11-27 18:45:00
ABJ    1984-01-01 00:00:00
ABR    1984-01-01 00:00:00
ADW    1984-01-01 00:00:00
               ...        
VRC    1993-09-23 22:20:00
VSP    1993-09-24 22:05:00
VWB    1985-01-01 00:00:00
WMP    1995-03-02 19:00:00
WWVB   1984-01-01 00:00:00
Length: 925, dtype: datetime64[ns]

The ``agg`` function allows to carry out several operations to each group of rows:

Select the stations that were deployed first and recovered last

In [40]:
data.groupby(['station']).agg({'start_time':lambda x: min(x), 'end_time':lambda x: max(x)})

,start_time,end_time
station,,
AAR,1984-01-01 00:00:00,2006-01-04 19:19:00
AAS,1984-11-27 18:45:00,2021-08-13 16:50:00
ABJ,1984-01-01 00:00:00,2019-06-26 19:17:00
ABR,1984-01-01 00:00:00,1997-08-04 21:02:00
ADW,1984-01-01 00:00:00,2006-04-20 01:08:00
...,...,...
VRC,1993-09-23 22:20:00,2001-07-12 16:50:00
VSP,1993-09-24 22:05:00,2001-07-12 16:50:00
VWB,1985-01-01 00:00:00,1985-03-25 19:00:00


We can also make groups by selecting the values of two categorical variables:

In [41]:
data.groupby(['station', 'channel']).agg({'start_time':lambda x: min(x), 'end_time':lambda x: max(x)})

start_time            end_time
station channel                                        
AAR     EHZ     1984-01-01 00:00:00 2006-01-04 19:19:00
        SHZ     1994-11-28 00:00:00 2006-01-04 19:19:00
AAS     EHZ     1984-11-27 18:45:00 2021-08-13 16:50:00
        SHZ     1994-11-28 00:00:00 2006-01-24 18:00:00
ABJ     EHZ     1984-01-01 00:00:00 2019-06-26 19:17:00
...                             ...                 ...
WMP     EHZ     1995-03-02 19:00:00 2002-05-08 22:30:00
        SHE     1995-07-02 12:00:00 2002-05-08 22:30:00
        SHN     1995-07-02 12:00:00 2002-05-08 22:30:00
        SHZ     1995-03-02 19:00:00 2002-05-08 22:30:00
WWVB    ATT     1984-01-01 00:00:00 1999-03-20 23:59:00

[4309 rows x 2 columns]

Previously, we just printed the output, but we can also store it in a new variable:

In [42]:
data_grouped = data.groupby(['station', 'channel']).agg({'start_time':lambda x: min(x), 'end_time':lambda x: max(x)})

Print the new dataframe and look at the rows indexes. Anything wrong?

In [54]:
# enter code here

0      1984-01-01 00:00:00
1      1987-05-01 00:00:00
2      1994-11-28 00:00:00
3      1984-11-27 18:45:00
4      1987-05-01 00:00:00
               ...        
7151   1995-07-02 12:00:00
7152   1995-03-02 19:00:00
7153   1995-07-02 12:00:00
7154   1995-07-02 12:00:00
7155   1984-01-01 00:00:00
Name: start_time, Length: 7156, dtype: datetime64[ns]


When we select only some rows, the index is not automatically reset to start at 0. We can do it manually. Many functions in pandas have also an option to reset the index, and option to transform the dataframe in place, instead of saving the results in another variable.

In [43]:
data_grouped.reset_index()

,station,channel,start_time,end_time
0,AAR,EHZ,1984-01-01 00:00:00,2006-01-04 19:19:00
1,AAR,SHZ,1994-11-28 00:00:00,2006-01-04 19:19:00
2,AAS,EHZ,1984-11-27 18:45:00,2021-08-13 16:50:00
3,AAS,SHZ,1994-11-28 00:00:00,2006-01-24 18:00:00
4,ABJ,EHZ,1984-01-01 00:00:00,2019-06-26 19:17:00
...,...,...,...,...
4304,WMP,EHZ,1995-03-02 19:00:00,2002-05-08 22:30:00
4305,WMP,SHE,1995-07-02 12:00:00,2002-05-08 22:30:00
4306,WMP,SHN,1995-07-02 12:00:00,2002-05-08 22:30:00
4307,WMP,SHZ,1995-03-02 19:00:00,2002-05-08 22:30:00


It is also possible to sort the dataset by value.

In [44]:
data_grouped.sort_values(by='start_time')

start_time            end_time
station channel                               
PG1     HV1     1983-06-27 1993-11-10 18:00:00
AAR     EHZ     1984-01-01 2006-01-04 19:19:00
HPL     ELE     1984-01-01 1996-05-30 23:59:00
        ELN     1984-01-01 1996-05-30 23:59:00
        ELZ     1984-01-01 1996-05-30 23:59:00
...                    ...                 ...
BTI     HNZ     2022-09-28 2025-01-01 00:00:00
        VDT     2022-09-28 2025-01-01 00:00:00
        VCO     2022-09-28 2025-01-01 00:00:00
        GLO     2022-09-28 2025-01-01 00:00:00
        LCE     2022-09-28 2025-01-01 00:00:00

[4309 rows x 2 columns]

We can apply the sorting to several columns:

In [ ]:
data_grouped.sort_values(by=['start_time', 'end_time'])

start_time            end_time
station channel                                        
PG1     HV1     1983-06-27 00:00:00 1993-11-10 18:00:00
PGC     EHZ     1984-01-01 00:00:00 1984-04-12 18:00:00
AFO     EHZ     1984-01-01 00:00:00 1984-04-23 00:00:00
MCH     EHZ     1984-01-01 00:00:00 1984-08-01 19:31:00
MCN     EHZ     1984-01-01 00:00:00 1984-09-22 16:45:00
...                             ...                 ...
PPBB    VEI     2020-11-19 00:00:00 2025-01-01 00:00:00
        VPB     2020-11-19 00:00:00 2025-01-01 00:00:00
KBN     HNE     2021-02-26 00:48:00 2025-01-01 00:00:00
        HNN     2021-02-26 00:48:00 2025-01-01 00:00:00
        HNZ     2021-02-26 00:48:00 2025-01-01 00:00:00

[3725 rows x 2 columns]

The ``merge`` function allows to merge two dataframes that have some but not all columns in common.


Now, we are going to merge this seismic metadata frame to the one from the Pacific Northwest Seismic Network and other stations that were deployed in the area. The file is a CSV file. Read the file and describe the data



In [71]:
# enter response here
data_pnw=pd.read_csv('PNSN_meta.csv')

In [72]:
data_pnw.head()

,channel_id,network,station,location,channel,latitude,longitude,elevation,depth,starttime,endtime,sampling_rate,azimuth
0,1,1D,4001,--,DPZ,46.1666,-122.189,0.0,0.0,1.388534e+09,1.418429e+09,250.0,NaN
1,2,1D,4003,--,DPZ,46.1665,-122.184,0.0,0.0,1.388534e+09,1.418429e+09,250.0,NaN
2,3,1D,4004,--,DPZ,46.1646,-122.182,402.4,0.0,1.388534e+09,1.418429e+09,250.0,NaN
3,4,1D,4005,--,DPZ,46.1627,-122.182,1276.2,0.0,1.388534e+09,1.418429e+09,250.0,NaN
4,5,1D,4006,--,DPZ,46.1609,-122.180,0.0,0.0,1.388534e+09,1.418429e+09,250.0,NaN


You will notice that this file contains odd times for starttime and end times. The keys are also different from the previous dataframe. In order to compare them and concatenate both metadata, we want to at least have common keys.

First, make sure that both dataframes have common keys. Use the function [``rename``](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) from ``pandas`` to rename the keys in data_pnw.

In [51]:
# enter answer below

Then convert the time stamps of start_time and end_time into date time objects. 

In [101]:

for icrap in len(data_pnw['starttime'])
datetime.fromtimestamp(data_pnw['starttime'][:])
# print(data_pnw.starttime)

# startdate = pd.to_timedelta(data_pnw['starttime'])
# startdate = pd.to_timedelta(datetime.fromtimestamp(data_pnw.starttime), format='%Y/%m/%d,%H:%M:%S')
# print(startdate)

TypeError: cannot convert the series to <class 'int'>

In [ ]:
df = pd.concat([data,data_pnw],ignore_index=True)

In [ ]:
# Earthquakes in filtered 2007-2009 catalog but not in (unfiltered) 2004-2011 catalog
df_all = pd.concat([df2, df1_filter], ignore_index=True)
df_merge = df_all.merge(df2.drop_duplicates(), on=['date'], how='left', indicator=True)
df_added_1 = df_merge[df_merge['_merge'] == 'left_only']

# Earthquakes in filtered 2004-2011 catalog but not in (unfiltered) 2007-2009 catalog
df_all = pd.concat([df1, df2_filter], ignore_index=True)
df_merge = df_all.merge(df1.drop_duplicates(), on=['date'], how='left', indicator=True)
df_added_2 = df_merge[df_merge['_merge'] == 'left_only']

### Overview of Earthquake catalog

In [ ]:
quake=pd.read_csv('Global_Quakes_IRIS.csv')
quake.head()

,Unnamed: 0,time,latitude,longitude,depth,magnitude,description
0,1784,2010-07-02 06:04:03.570,-13.6098,166.6541,34400.0,6.3,VANUATU ISLANDS
1,1783,2010-07-04 21:55:52.370,39.6611,142.5792,30100.0,6.3,"NEAR EAST COAST OF HONSHU, JAPAN"
2,1782,2010-07-10 11:43:33.000,11.1664,146.0823,16900.0,6.3,SOUTH OF MARIANA ISLANDS
3,1781,2010-07-12 00:11:20.060,-22.2789,-68.3159,109400.0,6.2,NORTHERN CHILE
4,1780,2010-07-14 08:32:21.850,-38.0635,-73.4649,25700.0,6.6,NEAR COAST OF CENTRAL CHILE


* Plot the earthquakes using Plotly functionality.

Use "description" as a hover name. 

Use "marker_size" for size and color the markers using the key 'magnitude'.

Use "magnitude" and "depth" as hover data.

In [ ]:
# enter answer here

Using ``matplotlib``, make a histogram of the magnitude distribution of these earthquakes. Find a way to plot the y-axis in log scale.

In [ ]:
# enter answer here